# Zillow Database Construction Guide

By Brian Truong, Fall 2018
#### In this guide I will demonstrate how to construct a Zillow database, join Zillow datasets, and more using terminal and sqlite. I will be working with California data in this guide as well as using the supercomputers on the EML servers.¶

## Terminal Lines

1) Open terminal and go to your scratch account on the EML server: 

    cd /scratch/public/account_name
    
2) Move the Zillow data that you need in /scratch/public/ztrax to your directory /scratch/public/account_name

3) Optional: Use this to reduce the size of your zillow data based on your needs. For instance, I used this to get the last half of the CA Zillow text data since the data is organized alphabetically by county, and San Mateo and Santa Clara are in the last half of the file.

    head -n zillow_data.txt > your_file.txt

4) Create your sqlite database:

    sqlite3 db_name.sqlite

### SQLite Lines

1) Create your SQL tables in the prompt from above:

> CREATE TABLE sm_trans_main_v1( TransId bigint(8), FIPS3 char(5), State3 char(2), County3 varchar(50), DataClassStndCode char(1), RecordTypeStndCode char(1), RecordingDate date(3), RecordingDocumentNumber varchar(25), RecordingBookNumber varchar(10), RecordingPageNumber varchar(10), ReRecordedCorrectionStndCode char(1), PriorRecordingDate date(3), PriorDocumentDate date(3), PriorDocumentNumber varchar(25), PriorBookNumber varchar(10), PriorPageNumber varchar(10), DocumentTypeStndCode char(4), DocumentDate date(3), SignatureDate date(3), EffectiveDate date(3), BuyerVestingStndCode char(2), BuyerMultiVestingFlag char(1), PartialInterestTransferStndCode char(1), PartialInterestTransferPercent decimal(5,3), SalesPriceAmount money(8), SalesPriceAmountStndCode char(2), CityTransferTax money(8), County3TransferTax money(8), State3TransferTax money(8), TotalTransferTax money(8), IntraFamilyTransferFlag char(1), TransferTaxExemptFlag char(1), PropertyUseStndCode char(2), AssessmentLandUseStndCode varchar(5), OccupancyStatusStndCode char(1), LegalStndCode char(1), BorrowerVestingStndCode char(2), LenderName varchar(250), LenderTypeStndCode char(2), LenderIDStndCode varchar(6), LenderDBAName varchar(135), DBALenderTypeStndCode char(2), DBALenderIDStndCode varchar(6), LenderMailCareOfName varchar(250), LenderMailHouseNumber varchar(13), LenderMailHouseNumberExt varchar(10), LenderMailStreetPreDirectional char(2), LenderMailStreetName varchar(50), LenderMailStreetSuffix varchar(6), LenderMailStreetPostDirectional char(2), LenderMailFullStreetAddress varchar(80), LenderMailBuildingName varchar(45), LenderMailBuildingNumber varchar(15), LenderMailUnitDesignator varchar(10), LenderMailUnit varchar(15), LenderMailCity varchar(45), LenderMailState3 char(2), LenderMailZip varchar(5), LenderMailZip4 char(4), LoanAmount money(8), LoanAmountStndCode char(1), MaximumLoanAmount money(8), LoanTypeStndCode char(2), LoanTypeClosedOpenEndStndCode char(1), LoanTypeFutureAdvanceFlag char(1), LoanTypeProgramStndCode char(2), LoanRateTypeStndCode char(3), LoanDueDate date(3), LoanTermMonths int(4), LoanTermYears int(4), InitialInterestRate decimal(5,3), ARMFirstAdjustmentDate date(3), ARMFirstAdjustmentMaxRate decimal(5,3), ARMFirstAdjustmentMinRate decimal(5,3), ARMIndexStndCode varchar(5), ARMAdjustmentFrequencyStndCode char(1), ARMMargin decimal(5,3), ARMInitialCap decimal(5,3), ARMPeriodicCap decimal(5,3), ARMLifetimeCap decimal(5,3), ARMMaxInterestRate decimal(5,3), ARMMinInterestRate decimal(5,3), InterestOnlyFlag char(1), InterestOnlyTerm char(3), PrepaymentPenaltyFlag char(1), PrepaymentPenaltyTerm char(3), BiWeeklyPaymentFlag char(1), AssumabilityRiderFlag char(1), BalloonRiderFlag char(1), CondominiumRiderFlag char(1), PlannedUnitDevelopmentRiderFlag char(1), SecondHomeRiderFlag char(1), OneToFourFamilyRiderFlag char(1), ConcurrentMtgeDocOrBkPg varchar(25), LoanNumber varchar(45), MERSMINNumber varchar(25), CaseNumber varchar(45), MERSFlag char(1), TitleCompanyName varchar(250), TitleCompanyIDStndCode varchar(6), AccommodationRecordingFlag char(1), UnpaidBalance money(8), InstallmentAmount money(8), InstallmentDueDate date(3), TotalDelinquentAmount money(8), DelinquentAsOfDate date(3), CurrentLender varchar(250), CurrentLenderTypeStndCode char(2), CurrentLenderIDStndCode varchar(6), TrusteeSaleNumber varchar(25), AttorneyFileNumber varchar(25), AuctionDate date(3), AuctionTime time(5), AuctionFullStreetAddress varchar(80), AuctionCityName varchar(45), StartingBid money(8), KeyedDate date(3), KeyerID varchar(10), SubVendorStndCode varchar(5), ImageFileName varchar(250), BuilderFlag char(1), MatchStndCode char(2), REOStndCode varchar(6), UpdateOwnershipFlag char(1), LoadID3 bigint(8), StatusInd int(4), TransactionTypeStndCode char(1), BatchID3 int(4), BKFSPID int(4), ZVendorStndCode char(3), SourceChkSum int(4));

**Note**: 
- SalePriceAmount, RecordingDate, PropertyUseStndCode, and AssessmentLandUseStndCode are the needed columns in Transaction Main

- PropertyUseStndCode and AssessmentLandUseStndCode will help us identify condos


2) Read in your zillow text file to your table:

> .separator '|'

> .import zillow_data.txt zillow_table

3) You can check the counties in your tables using:

    select distinct County3 from sm_trans_main_v1;

4) Optional: Get the county you need

    create table sm_trans_main_v2 as select * from sm_trans_main_v1 where County3 = 'SAN MATEO';


### Joining Tables

In this example I join the PropertyInfo, BuyerDescription, and the ForeclosureNameAddress tables with the Transaction Main. I add suffixes such as _P for PropertyInfo in order avoid name conflicts. The goal is to create a table of Transaction Main, PropertyInfo, BuyerDescription, and the ForeclosureNameAddress. Each of these tables contain certain columns that we need for our project so we consolidate them into one single table.

5) Creating another table for PropertyInfo.txt:

> CREATE TABLE trans_prop( TransId_P bigint(8), AssessorParcelNumber_P varchar(50), APNIndicatorStndCode_P char(1), TaxIDNumber_P varchar(50), TaxIDIndicatorStndCode_P char(1), UnformattedAssessorParcelNumber_P varchar(50), AlternateParcelNumber_P varchar(50), HawaiiCondoCPRCode_P char(4), PropertyHouseNumber_P varchar(13), PropertyHouseNumberExt_P varchar(10), PropertyStreetPreDirectional_P char(2), PropertyStreetName_P varchar(50), PropertyStreetSuffix_P varchar(6), PropertyStreetPostDirectional_P char(2), PropertyBuildingNumber_P varchar(45), PropertyFullStreetAddress_P varchar(80), PropertyCity_P varchar(45), PropertyState_P char(2), PropertyZip_P varchar(5), PropertyZip4_P char(4), OriginalPropertyFullStreetAddress_P varchar(100), OriginalPropertyAddressLastline_P varchar(100), PropertyAddressStndCode_P char(1), LegalLot_P varchar(100), LegalOtherLot_P varchar(100), LegalLotCode_P char(2), LegalBlock_P varchar(50), LegalSubdivisionName_P varchar(200), LegalCondoProjectPUDDevName_P varchar(100), LegalBuildingNumber_P varchar(45), LegalUnit_P varchar(100), LegalSection_P varchar(100), LegalPhase_P varchar(50), LegalTract_P varchar(50), LegalDistrict_P varchar(45), LegalMunicipality_P varchar(50), LegalCity_P varchar(50), LegalTownship_P varchar(50), LegalSTRSection_P varchar(100), LegalSTRTownship_P varchar(15), LegalSTRRange_P varchar(15), LegalSTRMeridian_P varchar(35), LegalSecTwnRngMer_P varchar(165), LegalRecordersMapReference_P varchar(100), LegalDescription_P varchar(2000), LegalLotSize_P varchar(14), PropertySequenceNumber_P int(4), PropertyAddressMatchcode_P char(1), PropertyAddressUnitDesignator_P varchar(10), PropertyAddressUnitNumber_P varchar(11), PropertyAddressCarrierRoute_P char(4), PropertyAddressGeoCodeMatchCode_P char(1), PropertyAddressLatitude_P decimal(9,6), PropertyAddressLongitude_P decimal(9,6), PropertyAddressCensusTractAndBlock_P varchar(16), PropertyAddressConfidenceScore_P tinyint(1), PropertyAddressCBSACode_P int(4), PropertyAddressCBSADivisionCode_P int(4), PropertyAddressMatchType_P tinyint(1), PropertyAddressDPV_P char(1), PropertyGeocodeQualityCode_P varchar(10), PropertyAddressQualityCode_P varchar(10), FIPS_P char(5), LoadID_P bigint(8), ImportParcelID_P bigint(8), BKFSPID_P int(4), AssessmentRecordMatchFlag_P tinyint(1), BatchID_P int(4));

**Note:** 
- PropertyFullStreetAddress, PropertyZip, AssessorParcelNumber, and ImportParcelID are the columns needed in PropertyInfo
- TransId is used to link both datasets
- ImportParcelID will help us group unique parcels across varying date ranges
- AssessorParcelNumber will be used for our pivot query in order to have each property as its own row

6) Joining trans_prop and sm_trans_main_v2
> create table sm_trans_main_prop as select * from sm_trans_main_v2 inner join trans_prop on trans_prop.TransId_P = sm_trans_main_v2.TransId;

7) Joining sm_trans_main_prop and trans_buyer: 
> create table sm_trans_main_prop_buyer as select * from sm_trans_main_prop_v2 left join trans_buyer on sm_trans_main_prop_v2.TransId = trans_buyer.TransId_B;

**Note:**

- TransId is used to link both datasets

- We add in BuyerDescriptionCodeWe because we want to know what kind of sale each transaction is. This will help us identify if the transactions with 0s on the sales are foreclosures, refinancing, etc. 

- A left join is used to avoid losing data from the main core of the data: Transaction Main and PropertyInfo. If we used an inner join then it would look for every match within both datasets. Some data in Transaction Main and PropertyInfo may not be present in the other dataset which means that we will lose data on the Transaction Main and PropertyInfo. Thus we use a left join to avoid this.


8) Joining sm_trans_main_prop_buyer and trans_foreclosure
> create table sm_trans_main_prop_buyer_foreclosure as select * from sm_trans_main_prop_buyer left join trans_foreclosure on sm_trans_main_prop_buyer.TransId = trans_foreclosure.TransId_F;

**Note:**
- Join the table ForeclosureNameAddress to the dataset. So whichever transaction for a property that has a non-empty TransId_Foreclosure column value should indicate a foreclosure.
- The suffix _P, _B, _F are added to PropertyInfo, Buyer, and Foreclosure respectively in order to avoid name collisions when joining tables.


9) Select relevant columns. These are the columns that we need for our project. Each of the reasoning is described aboved in the notes: 
> create table sm_trans_main_prop_buyer_foreclosure_v2 as select TransId, SalesPriceAmount, RecordingDate, PropertyFullStreetAddress_P, PropertyZip_P, AssessorParcelNumber_P, ImportParcelID_P, PropertyUseStndCode, AssessmentLandUseStndCode, TransId_F, BuyerDescriptionStndCode_B from sm_trans_main_prop_buyer_foreclosure;

10) Convert to csv and use this for the pivot query
.header on

.mode csv

.once /scratch/public/account/sm_trans_main_prop_buyer_foreclosure_v2.csv

select * sm_trans_main_prop_buyer_foreclosure_v2;


**OPTIONAL (JOINING WITH ASSESSOR MAIN):**

We might need this for later down the road so I’m writing this in case.

11) Attaching Assessor Main for SM, since Assessor Main is another sqlite database. We will join this with the transaction database we've create.
>  attach 'ZMainBldgCA.sqlite' as zmb;

Optional: Save zmb
>  create table sm_assr_main as select * from zmb.san_mateo;

12) For grouping unique record among different years of a parcel for the newest record.
> create table sm_assr_main_u as select x.RowID, x.ImportParcelID, x.LoadID from (select *, max(LoadID) as NewestLoadID from sm_assr_main group by ImportParcelID) as x inner join sm_assr_main as a on a.ImportParcelID = x.ImportParcelID and a.LoadID = x.NewestLoadID;

13) Joining the transaction and assessor databases. They have ImportParcelID in common. 
> create table  sm_trans_main_prop_buyer_foreclosure_assr_main  as select * from sm_trans_main_prop inner join zmb.san_mateo on sm_trans_main_prop_buyer_foreclosure_assr_main.ImportParcelID_P = zmb.san_mateo.ImportParcelID;



### Static Pivot Query

A Pivot Query is used in order to group each property into a row, while transforming the transaction, buyer, and foreclosure data into sequenced columns. This will help reduce repetition.

Please check my pivot query notebook for this: 

Link to be added
